## Model 1

In [ ]:
from scipy.optimize import linprog

def solve_transportation_problem(costs, supplies, demands):
    m = len(supplies)
    n = len(demands)

    # Create the cost matrix
    c = np.array([costs[i][j] for i in range(m) for j in range(n)])

    # Create the equality constraint matrix
    A_eq = []
    for i in range(m):
        row = [int(j == i) for j in range(m)]
        row.extend([0] * n)
        A_eq.append(row)
    for j in range(n):
        row = [0] * m
        row.extend([int(i == j) for i in range(n)])
        A_eq.append(row)
    b_eq = supplies + demands

    # Create the inequality constraint matrices
    A_ub = []
    for i in range(m):
        row = [1 if j == i else 0 for j in range(m)]
        row.extend([0] * n)
        A_ub.append(row)
    b_ub = supplies

    A_lb = []
    for j in range(n):
        row = [0] * m
        row.extend([-1 if i == j else 0 for i in range(n)])
        A_lb.append(row)
    b_lb = [-demand for demand in demands]

    res = linprog(-c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0, None))

    return res.x

In [ ]:
costs = [[15, 13], [15, 18]]
supplies = [1600, 1600]
demands = [1555, 1575]

solution = solve_transportation_problem(costs, supplies, demands)
print(solution)

[1600. 1600. 1555. 1575.]


## Model 2

In [ ]:
from scipy.optimize import linprog
import numpy as np

def solve_multimodal_transportation_problem(cost_matrices, availabilities, demands, storages):
    num_modes = len(cost_matrices)
    m = len(availabilities[0])  # Number of suppliers
    n = len(demands[0])         # Number of consumers

    # Constructing the objective function coefficients
    c = np.zeros((m * n * num_modes,))
    for r in range(num_modes):
        for i in range(m):
            for j in range(n):
                c[r * m * n + i * n + j] = cost_matrices[r][i][j]

    # Constructing equality constraints
    A_eq = []
    b_eq = []
    for r in range(num_modes):
        for i in range(m):
            for j in range(n):
                eq_constraint = np.zeros((num_modes * m * n,))
                eq_constraint[r * m * n + i * n + j] = 1
                A_eq.append(eq_constraint)
                b_eq.append(availabilities[r][i][j])
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)

    # Constructing inequality constraints
    A_ub = np.zeros((n, num_modes * m * n))
    b_ub = np.zeros((n,))
    for r in range(num_modes):
        for i in range(m):
            for j in range(n):
                A_ub[j, r * m * n + i * n + j] = 1
                b_ub[j] = storages[r][i][j]

    # Demand constraints
    for j in range(n):
        for r in range(num_modes):
            for i in range(m):
                A_ub = np.vstack((A_ub, np.zeros((1, num_modes * m * n))))
                A_ub[-1, r * m * n + i * n + j] = 1
                b_ub = np.concatenate((b_ub, [demands[0][j]]))

    # Bounds
    bounds = [(0, None)] * (m * n * num_modes)

    # Solve the linear programming problem
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

    if res.success:
        return res.x.reshape((num_modes, m, n))
    else:
        print("Optimization failed: ", res.message)
        return None

In [ ]:
cost_matrices = [
    [
        [10, 15, 20],
        [25, 18, 12],
        [30, 25, 16]
    ],
    [
        [12, 17, 22],
        [27, 20, 14],
        [32, 27, 18]
    ]
]

availabilities = [
    [
        [50, 60, 70],
        [40, 55, 65],
        [30, 45, 55]
    ],
    [
        [55, 65, 75],
        [45, 60, 70],
        [35, 50, 60]
    ]
]

demands = [[30, 40, 50]]

storages = [
    [
        [20, 25, 30],
        [18, 23, 28],
        [16, 21, 26]
    ],
    [
        [22, 27, 32],
        [20, 25, 30],
        [18, 23, 28]
    ]
]

solution = solve_multimodal_transportation_problem(cost_matrices, availabilities, demands, storages)
print(solution)

Optimization failed:  The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)
None


## Model 4

In [ ]:
import torch
import torch.nn as nn

class MultiLayerPerceptron(nn.Module):
    def __init__(self):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = nn.Linear(3, 5)  # Input layer (3 nodes) to hidden layer (5 nodes)
        self.fc2 = nn.Linear(5, 6)  # Hidden layer (5 nodes) to output layer (6 nodes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def loss_function(output, target):
    z1 = 0.7 * output[:, 0] + 0.95 * output[:, 1] + 0.95 * output[:, 2] + 0.2 * output[:, 3] + 0.25 * output[:, 4]
    z2 = 0.35 * output[:, 0] + 0.45 * output[:, 1] + 0.48 * output[:, 2] + 0.44 * output[:, 3] + 0.5 * output[:, 4] + 0.6 * output[:, 5]
    z3 = 0.4 * output[:, 0] + 0.5 * output[:, 1] + 0.6 * output[:, 2] + 0.5 * output[:, 3] + 0.5 * output[:, 4] + 0.8 * output[:, 5]
    z = z1 + z2 + z3

    return z.mean()

# Create the neural network
model = MultiLayerPerceptron()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(500):
    # Generate input data
    input_data = torch.randn(1, 3)  # Batch size of 1, input size of 3

    # Forward pass
    output = model(input_data)

    # Calculate the loss
    loss = loss_function(output, input_data)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 100 epochs
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 50, Loss: -15.099015235900879
Epoch 100, Loss: -55.54655075073242
Epoch 150, Loss: -182.94483947753906
Epoch 200, Loss: -219.26260375976562
Epoch 250, Loss: -273.86029052734375
Epoch 300, Loss: -617.5321655273438
Epoch 350, Loss: -1131.4298095703125
Epoch 400, Loss: -1424.510986328125
Epoch 450, Loss: -1985.369140625
Epoch 500, Loss: -2718.721435546875


In [ ]:
import torch

# Example input data
input_data = torch.tensor([[0.1, 0.2, 0.3]])

# Create the neural network
model = MultiLayerPerceptron()

# Forward pass
output = model(input_data)
print("Output:", output)

# Calculate the loss
loss = loss_function(output, input_data)
print("Loss:", loss.item())

Output: tensor([[-0.2044,  0.0591, -0.1949,  0.2098,  0.1841,  0.0188]],
       grad_fn=<AddmmBackward0>)
Loss: -0.08412109315395355


## Generalized MLP solving the example problem done above

In [ ]:
import tensorflow as tf
import numpy as np

# Define the MultiLayerPerceptron class
class MultiLayerPerceptron(tf.keras.Model):
    def __init__(self, input_dim, hidden_units, output_dim):
        super(MultiLayerPerceptron, self).__init__()
        self.fc1 = tf.keras.layers.Dense(hidden_units, activation='relu', input_dim=input_dim)
        self.fc2 = tf.keras.layers.Dense(output_dim)

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        return x

# Define the loss function
def loss_function(output, target):
    loss = tf.reduce_mean(tf.square(output - target))
    return loss

# Sample input data
cost_matrices = [
    np.array([
        [10, 15, 20],
        [25, 18, 12],
        [30, 25, 16]
    ]),
    np.array([
        [12, 17, 22],
        [27, 20, 14],
        [32, 27, 18]
    ])
]

availabilities = [
    np.array([
        [50, 60, 70],
        [40, 55, 65],
        [30, 45, 55]
    ]),
    np.array([
        [55, 65, 75],
        [45, 60, 70],
        [35, 50, 60]
    ])
]

demands = np.array([[30, 40, 50]])

storages = [
    np.array([
        [20, 25, 30],
        [18, 23, 28],
        [16, 21, 26]
    ]),
    np.array([
        [22, 27, 32],
        [20, 25, 30],
        [18, 23, 28]
    ])
]

# Concatenate all input data along the first axis
input_data = np.concatenate(cost_matrices + availabilities + storages, axis=0)
# Define model parameters
input_dim = input_data.shape[-1]  # Dimension of input data
hidden_units = 10  # Number of units in the hidden layer
output_dim = 1  # Dimension of output data (for simplicity, we'll predict a single value)
# Create the neural network
model = MultiLayerPerceptron(input_dim, hidden_units, output_dim)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Training loop
for epoch in range(500):
    # Generate target data (replace this with your actual target data)
    target_data = np.random.rand(1, output_dim)  # Example: random target data for illustration purposes

    # Forward pass
    with tf.GradientTape() as tape:
        output = model(input_data)
        loss = loss_function(output, target_data)

    # Backward pass
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Print the loss every 50 epochs
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")


Epoch 50, Loss: 2.6839566230773926
Epoch 100, Loss: 1.1491353511810303
Epoch 150, Loss: 0.9161852598190308
Epoch 200, Loss: 0.8773241639137268
Epoch 250, Loss: 0.8803513050079346
Epoch 300, Loss: 0.6596843004226685
Epoch 350, Loss: 0.5993877053260803
Epoch 400, Loss: 0.6293603181838989
Epoch 450, Loss: 0.4517744779586792
Epoch 500, Loss: 0.35894331336021423


In [ ]:
# Generate some test examples
num_examples = 5
test_examples = np.random.randn(num_examples, input_dim)  # Generating random test examples

# Use the trained model for predictions
for i, example in enumerate(test_examples):
    prediction = model(example[np.newaxis, :])  # Forward pass through the model
    print(f"Test Example {i+1}: Input = {example}, Prediction = {prediction.numpy()[0][0]}")

Test Example 1: Input = [ 1.69966926  2.23410743 -0.20023704], Prediction = 0.3583950400352478
Test Example 2: Input = [-1.68952781  1.68726399  0.20395108], Prediction = 0.46149539947509766
Test Example 3: Input = [-1.39273888  0.88300849 -1.44773798], Prediction = 0.6194852590560913
Test Example 4: Input = [-0.97206335  1.06258302  0.65960994], Prediction = 0.21457673609256744
Test Example 5: Input = [ 0.67602809  0.54130242 -1.45443793], Prediction = -0.10116668045520782
